In [73]:
import polars as pl
import numpy as np

data_path = "data"

In [74]:
sales = (
    pl.read_csv("../data/sales_train_evaluation.csv")
    .lazy()
    .melt(id_vars=[
        "id", "item_id", "dept_id", "cat_id", "store_id", "state_id"
        ], value_name="sales", variable_name="d")
    .with_columns(
        pl.col("d").str.slice(2, ).alias("days_since_start").cast(int)
    )
)

In [75]:
calendar = (
    pl.read_csv("../data/calendar.csv")
    .lazy()
    .melt(id_vars=[
        "date", "wm_yr_wk", "weekday", "wday", "month", "year", "d", 
        "event_name_1", "event_type_1", "event_name_2", "event_type_2"
        ], variable_name="state_id", value_name="snap")
    .with_columns(
        pl.col("date").str.strptime(pl.Date, "%Y-%m-%d"),
        pl.col("state_id").str.slice(-2, ),
        pl.concat_str(pl.col("event_name_1").str.replace("\s", "_"), pl.col("event_type_1").str.replace("\s", "_"), separator="/").alias("event_1"),
        pl.concat_str(pl.col("event_name_2").str.replace("\s", "_"), pl.col("event_type_2").str.replace("\s", "_"), separator=" / ").alias("event_2")
    )
    .with_columns(
        pl.concat_str(pl.col("event_1"), pl.col("event_2"), separator=" ").fill_null("").alias("event")
    )
    .drop(["event_name_1", "event_type_1", "event_name_2", "event_type_2", "event_1", "event_2"])
)

In [77]:
calendar.head().collect()

date,wm_yr_wk,weekday,wday,month,year,d,state_id,snap,event
date,i64,str,i64,i64,i64,str,str,i64,str
2011-01-29,11101,"""Saturday""",1,1,2011,"""d_1""","""CA""",0,""""""
2011-01-30,11101,"""Sunday""",2,1,2011,"""d_2""","""CA""",0,""""""
2011-01-31,11101,"""Monday""",3,1,2011,"""d_3""","""CA""",0,""""""
2011-02-01,11101,"""Tuesday""",4,2,2011,"""d_4""","""CA""",1,""""""
2011-02-02,11101,"""Wednesday""",5,2,2011,"""d_5""","""CA""",1,""""""


In [66]:
prices = (
    pl.read_csv("../data/sell_prices.csv")
    .lazy()
)

In [67]:
df = (
    sales
    .join(calendar, on=["d", "state_id"])
    # NOTE: if the price is not reported in the prices table this filters automatically the
    #       days without listings in the beginning
    .join(prices, on=["store_id", "item_id", "wm_yr_wk"])
    .collect()
)

In [69]:
df.head()

id,item_id,dept_id,cat_id,store_id,state_id,d,sales,days_since_start,date,wm_yr_wk,weekday,wday,month,year,snap,event,sell_price
str,str,str,str,str,str,str,i64,i64,date,i64,str,i64,i64,i64,i64,str,f64
"""HOBBIES_1_008_…","""HOBBIES_1_008""","""HOBBIES_1""","""HOBBIES""","""CA_1""","""CA""","""d_1""",12,1,2011-01-29,11101,"""Saturday""",1,1,2011,0,"""""",0.46
"""HOBBIES_1_009_…","""HOBBIES_1_009""","""HOBBIES_1""","""HOBBIES""","""CA_1""","""CA""","""d_1""",2,1,2011-01-29,11101,"""Saturday""",1,1,2011,0,"""""",1.56
"""HOBBIES_1_010_…","""HOBBIES_1_010""","""HOBBIES_1""","""HOBBIES""","""CA_1""","""CA""","""d_1""",0,1,2011-01-29,11101,"""Saturday""",1,1,2011,0,"""""",3.17
"""HOBBIES_1_012_…","""HOBBIES_1_012""","""HOBBIES_1""","""HOBBIES""","""CA_1""","""CA""","""d_1""",0,1,2011-01-29,11101,"""Saturday""",1,1,2011,0,"""""",5.98
"""HOBBIES_1_015_…","""HOBBIES_1_015""","""HOBBIES_1""","""HOBBIES""","""CA_1""","""CA""","""d_1""",4,1,2011-01-29,11101,"""Saturday""",1,1,2011,0,"""""",0.7


In [70]:
df.write_parquet("../data/full_table.parquet")

In [71]:
(
    df
    .select([
        "id", "item_id", "dept_id", "cat_id", "store_id", "state_id", 
        "sales", "date", "event", "snap", "sell_price"
    ])
    .write_parquet("../data/reduced_table.parquet")
)